In [40]:
import multiprocessing as mp
import typing
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import matplotlib.ticker
import numpy as np
import scipy as sp
import sklearn.utils
from tqdm import tqdm
import pandas as pd

from IPython import get_ipython  # For automatically-generated python file.

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
def get_random_number_for_right_deck(n: int, seed: int=None, ) -> int:
    """
    Return the number of cards to split into the right sub-deck.

    :param n: one above the highest number that could be returned by this
              function.
    :param seed: optional seed for the random number generator to enable
                 deterministic behavior.
    :return: a random integer (between 1 and n-1) that represents the
             desired number of cards.

    Examples:

    >>> get_random_number_for_right_deck(n=5, seed=0, )
    1
    """
    random = sklearn.utils.check_random_state(seed=seed, )
    
    return random.randint(low=1, high=n, )

In [4]:
def should_drop_from_right_deck(n_left: int, n_right:int, seed: int=None, ) -> bool:
    """
    Determine whether we drop a card from the right or left sub-deck.
    
    Either `n_left` or `n_right` (or both) must be greater than zero.
    
    :param n_left: the number of cards in the left sub-deck.
    :param n_right: the number of cards in the right sub-deck.
    :param seed: optional seed for the random number generator to
                 enable deterministic behavior.
    :return: True if we should drop a card from the right sub-deck,
             False otherwise.
    
    Examples:

    >>> should_drop_from_right_deck(n_left=32, n_right=5, seed=0, )
    True

    >>> should_drop_from_right_deck(n_left=0, n_right=5, )
    True

    >>> should_drop_from_right_deck(n_left=7, n_right=0, )
    False

    >>> should_drop_from_right_deck(n_left=0, n_right=0, )
    Traceback (most recent call last):
    ...
    ValueError: Either `n_left` or `n_right` (or both) must be greater than zero.
    """
    if n_left > 0 and n_right > 0:
        # There are cards left in both sub-decks, so pick a
        # sub-deck at random.
        random = sklearn.utils.check_random_state(seed=seed, )
        num = random.randint(low=0, high=2, )
        boolean = (num == 0)
        return boolean
    elif n_left == 0 and n_right > 0:
        # There are no more cards in the left sub-deck, only
        # the right sub-deck, so we drop from the right sub-deck.
        return True
    elif n_left > 0 and n_right == 0:
        # There are no more cards in the right sub-deck, only
        # the left sub-deck, so we drop from the left sub-deck.
        return False
    else:
        # There are no more cards in either sub-deck.
        raise ValueError ('Either `n_left` or `n_right` '\
                          '(or both) must be greater than zero.')

In [5]:
def shuffle(deck: np.array, seed: int=None, ) -> np.array:
    """
    Shuffle the input 'deck' using the Gilbert–Shannon–Reeds method.

    :param seq: the input sequence of integers.
    :param seed: optional seed for the random number generator
                 to enable deterministic behavior.
    :return: A new deck containing shuffled integers from the
             input deck.

    Examples:

    >>> shuffle(deck=np.array([0, 7, 3, 8, 4, 9, ]), seed=0, )
    array([4, 8, 3, 7, 0, 9])
    """
    
    # First randomly divide the 'deck' into 'left' and 'right'
    # 'sub-decks'.
    num_cards_in_deck = len(deck)
    orig_num_cards_right_deck = get_random_number_for_right_deck(
        n=num_cards_in_deck,
        seed=seed,
    )

    # By definition of get_random_number_for_right_deck():
    n_right = orig_num_cards_right_deck
    
    n_left = num_cards_in_deck - orig_num_cards_right_deck
    
    shuffled_deck = np.empty(num_cards_in_deck, dtype=int)
    
    # We will drop a card n times.
    for index in range(num_cards_in_deck):
        drop_from_right_deck = should_drop_from_right_deck(
            n_left=n_left,
            n_right=n_right,
            seed=seed,
        )
        
        if drop_from_right_deck is True:
            # Drop from the bottom of right sub-deck
            # onto the shuffled pile.
            shuffled_deck[index] = deck[n_right - 1]
            n_right = n_right - 1
        else:
            # Drop from the bottom of left sub-deck
            # onto the shuffled pile.
            shuffled_deck[index] = deck[
                orig_num_cards_right_deck + n_left - 1
            ]
            n_left = n_left - 1
    
    return shuffled_deck

In [6]:
num_cards = 52
max_num_shuffles = 20
num_decks = 10000

# Shuffling the cards using a uniform probability
# distribution results in the same expected frequency
# for each card in each deck position.
uniform_rel_freqs = np.full(
    shape=[num_cards, num_cards],
    fill_value=1./num_cards,
)

uniform_rel_freqs

array([[0.01923077, 0.01923077, 0.01923077, ..., 0.01923077, 0.01923077,
        0.01923077],
       [0.01923077, 0.01923077, 0.01923077, ..., 0.01923077, 0.01923077,
        0.01923077],
       [0.01923077, 0.01923077, 0.01923077, ..., 0.01923077, 0.01923077,
        0.01923077],
       ...,
       [0.01923077, 0.01923077, 0.01923077, ..., 0.01923077, 0.01923077,
        0.01923077],
       [0.01923077, 0.01923077, 0.01923077, ..., 0.01923077, 0.01923077,
        0.01923077],
       [0.01923077, 0.01923077, 0.01923077, ..., 0.01923077, 0.01923077,
        0.01923077]])

In [7]:
def calculate_differences(
    num_shuffles: int
    ) -> typing.Tuple[np.float64, np.float64, np.float64,]:
    """
    Calculate differences between observed and uniform distributions.
    
    :param The number of times to shuffle the deck each time.
    :return Three metrics for differences between the
            observed and uniform relative frequencies.
    """
    shuffled_decks = np.empty(shape=[num_decks, num_cards], )

    # First create a random deck.
    orig_deck = np.array(range(num_cards))
    np.random.shuffle(orig_deck)

    for i in range(num_decks):
        # Now shuffle this deck using the Gilbert–Shannon–Reeds method.
        new_deck = orig_deck
        for j in range(num_shuffles):
            new_deck = shuffle(new_deck)
        
        shuffled_decks[i] = new_deck

    # Calculate the relative frequencies of each card in each position.
    rel_freqs = np.empty(shape=[num_cards, num_cards], )

    for i in range(num_cards):
        col = shuffled_decks[:, i]

        # Make sure that each card appears at least once in this
        # position, by first adding the entire deck, and then
        # subtracting 1 from the total counts of each card in
        # this position.
        col = np.append(col, orig_deck)
        unique_values, counts = np.unique(col, return_counts=True)
        col_freqs = counts - 1
        rel_freqs[i] = col_freqs / num_decks

    # Here I use three metrics for differences between the
    # observed and uniform relative frequencies:
    # * The sum of the squared element-wise differences,
    # * The relative information entropy, and
    # * The Kolmogorov-Smirnov statistic.
    sum_squared = np.sum(np.square(np.subtract(uniform_rel_freqs, rel_freqs)))
    entropy = sp.stats.entropy(rel_freqs.flatten(), uniform_rel_freqs.flatten())
    kstest = sp.stats.kstest(rel_freqs.flatten(), 'uniform').statistic
    
    return sum_squared, entropy, kstest

In [8]:
results = []
max_num_shuffles = 20
for num_shuffles in tqdm(range(1, max_num_shuffles+1)):
    results.append(calculate_differences(num_shuffles))

# results = map(calculate_differences, range(1, max_num_shuffles+1))
# results = np.array(list(results))

results

100%|██████████| 20/20 [06:58<00:00, 20.90s/it]


[(4.9119175, 0.859852396350672, 0.8972136094674557),
 (0.59366016, 0.18538601831910645, 0.9153029585798816),
 (0.15744098, 0.05948242275702155, 0.9449562130177515),
 (0.05236916, 0.022288224909014834, 0.9566940828402366),
 (0.020702280000000003, 0.009623547283695438, 0.9657923076923077),
 (0.011246119999999998, 0.005458884071941429, 0.9685508875739646),
 (0.007351140000000001, 0.0036533445473508463, 0.9728508875739645),
 (0.0058022, 0.0028980494921471748, 0.9742603550295857),
 (0.0053313, 0.0026679163336771486, 0.9755301775147929),
 (0.0052075, 0.002610538563536007, 0.9761301775147929),
 (0.00509506, 0.002546824942789396, 0.9763),
 (0.004954179999999999, 0.002477744503408052, 0.9752),
 (0.00515034, 0.002574810581730702, 0.9759),
 (0.00498172, 0.0024980618395557324, 0.9758),
 (0.004981019999999999, 0.0024911375137805425, 0.9761301775147929),
 (0.0050455, 0.0025230205304473765, 0.9762301775147929),
 (0.00515002, 0.0025781242793385772, 0.9763),
 (0.00486228, 0.0024349264990087883, 0.9763)

In [41]:
sums_squared = [result[0] for result in results]
entropies = [result[1] for result in results]
kstests = [result[2] for result in results] #"of the most use"

In [56]:
fs = 14
fig, ax = plt.subplots(figsize=(8, 6), dpi=300)
ax.scatter(range(1, max_num_shuffles + 1), kstests, );
ax.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
ax.set_xlabel('Number of Shuffles', fontsize=fs, )
ax.set_ylabel('Kolmogorov-Smirnov Statistic', fontsize=fs, )
ax.set_xlim([0, max_num_shuffles + 1])
plt.show()
fig.savefig("kstests.png")

<IPython.core.display.Javascript object>

In [57]:
fig, ax = plt.subplots(figsize=(8, 6), dpi=300)
ax.scatter(range(1, max_num_shuffles + 1), sums_squared, );
ax.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
ax.set_xlabel('Number of Shuffles', fontsize=fs, )
ax.set_ylabel('Sum of the Squared Differences', fontsize=fs, )
ax.set_xlim([0, max_num_shuffles + 1])
plt.show();
fig.savefig("sums_squared.png")

<IPython.core.display.Javascript object>

In [58]:
fig, ax = plt.subplots(figsize=(8, 6), dpi=300)
ax.scatter(range(1, max_num_shuffles + 1), entropies, );
ax.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
ax.set_xlabel('Number of Shuffles', fontsize=fs, )
ax.set_ylabel('Relative Information Entropy', fontsize=fs, )
ax.set_xlim([0, max_num_shuffles + 1])
plt.show();
fig.savefig("entropies.png")

<IPython.core.display.Javascript object>

In [45]:
table = pd.DataFrame(index=range(1, max_num_shuffles + 1))
table['ks'] = kstests
table['sums_squared'] = sums_squared
table['entropies'] = entropies

diffs_array = [0]
for index in range(1, len(kstests)):
    diffs_array.append(kstests[index] - kstests[index-1])

table['benefit from adding this shuffle'] = diffs_array
table

,ks,sums_squared,entropies,benefit from adding this shuffle
1,0.897214,4.911918,0.859852,0.000000
2,0.915303,0.593660,0.185386,0.018089
3,0.944956,0.157441,0.059482,0.029653
4,0.956694,0.052369,0.022288,0.011738
5,0.965792,0.020702,0.009624,0.009098
6,0.968551,0.011246,0.005459,0.002759
7,0.972851,0.007351,0.003653,0.004300
8,0.974260,0.005802,0.002898,0.001409
9,0.975530,0.005331,0.002668,0.001270
10,0.976130,0.005208,0.002611,0.000600


In [59]:
fig, ax = plt.subplots(figsize=(8, 6), dpi=300)
ax.scatter(range(1, max_num_shuffles + 1), diffs_array, );
ax.xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
ax.set_xlabel('Number of Shuffles', fontsize=fs, )
ax.set_ylabel('Relative Information Entropy', fontsize=fs, )
ax.set_xlim([0, max_num_shuffles + 1])
plt.show();
fig.savefig("diffs.png")

<IPython.core.display.Javascript object>